# Simple neural networks with Keras

We will create neural netwokrs with the Keras framework, an API that controls other software capable to interact with the GPU of computers

As GPU software, we will use Tensorflow

To install Tensorflow on linux or Window, follow https://www.tensorflow.org/install

To install Tensorflow on Mac arm64 (M2 or higher), follow https://developer.apple.com/metal/tensorflow-plugin/

In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
# Check tensorflow configuration
import tensorflow as tf

tf.config.list_physical_devices('GPU')

In [ ]:
# import utilities from tensorflow

from tensorflow.keras.layers import Dense, Input, Flatten,\
                                    Reshape, LeakyReLU as LR,\
                                    Activation, Dropout, BatchNormalization

from tensorflow.keras.optimizers import SGD, Adam

from tensorflow.keras.models import Model, Sequential

from keras.callbacks import EarlyStopping

from tensorflow.keras.backend import clear_session

In [ ]:
# set random numbers for reproducibility

from numpy.random import seed
seed(123)
from tensorflow.keras.utils import set_random_seed
set_random_seed(2314)

In [ ]:
# import sklearn tools for commodity

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Regression with a multilayer perceptron

Simple curves can be fitted with linear or polynomial regression.

For more complex regressions, we can use a multilayer perceptron

## Synthetic example: Non-linear function

In [ ]:
# create synthetic datapoints with strong non-linearities

x = np.linspace(-3, 3, 200).reshape(-1, 1)
    
# Generate y using a complex nonlinear function: y = sin(X) + X^2/3 + noise
true_function = np.sin(x) + x**2/3
y = true_function.flatten() + np.random.normal(0, 0.1, 200)

plt.scatter(x, y,edgecolors='g')

plt.plot(x, true_function, "-r")

In [ ]:
# define our neural network using a sequential model in which we add layers 
# in sequence to the network architecture
# Let us start with a simple NN with a single hidden layer

# define the keras model

model = Sequential()

In [ ]:
# define the shape of the input layer (dimensio of the input data)
model.add(Input(shape=(1,)))

In [ ]:
# add the first hidden layer of 64 nodes, indicating that the number of features of the input is 1
# the NN has fully connected architecture, so we use the Dense object
# we choose 'tanh' as activation function, usually good for regression problems
model.add(Dense(64, activation='tanh'))

In [ ]:
## add the output layer with a single node
model.add(Dense(1))

In [ ]:
# compile the neural network
# for a regression, we use as loss (cost) function 'mse' (mean squared error)
# the optimizer (to minimize the cost function) is the stochastic gradient descent algorithm "adam"
# collect and record as metrics the mean absolute error "mae"

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

In [ ]:
# summary of the model
model.summary()

193 parameters to fit

In [ ]:
# train the neural network with the 'fit' method
# training takes places in epochs = number of passes of in the training dataset
# and batches = number of samples in the model withing an epoch in which the weights (parameters) of the NN are updated

# the fit method returns a history object that contains the loss and metrics values during training

EPOCHS = 100
history = model.fit(x, y, epochs=EPOCHS, batch_size=10)

In [ ]:
# get the preditions of the model
y_pred = model.predict(x)

In [ ]:
# plot the real data versus the predictions
plt.scatter(x, y, edgecolors='g', label="Real data")
plt.plot(x, y_pred, 'r', label="Predictions")
plt.legend()

The fit is not bad. 

Let us improve it with a more complex NN, and by tweaking some hyperparameters

In [ ]:
# clear the session and the previous NN defined
clear_session()

In [ ]:
# define the model all at once, increasing the number of hidden layers, and their size

model = Sequential()
model.add(Input(shape=(1,)))
model.add(Dense(128, activation='tanh'))
model.add(Dense(128, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(1))

In [ ]:
# Change the learning rate of the optimizer, making it take smaller steps in the
# search of the minimum of the loss function
adam = Adam(learning_rate=0.001)

model.compile(loss='mse', optimizer=adam,  metrics=['mae'])


In [ ]:
# summary of the model
model.summary()

Now 25000 parameters to fit

In [ ]:
# Train with more epochs, but increasing the batch size to speed up the process
EPOCHS = 500
history = model.fit(x, y, epochs=EPOCHS, batch_size=64)

In [ ]:
# get the preditions of the model
y_pred = model.predict(x)

# plot the real data versus the predictions
plt.scatter(x, y, edgecolors='g', label="Real data")
plt.plot(x, y_pred, 'r', label="Predictions")
plt.legend()

The fit is now very good!

We can try to improve it with more tweakings: Changes in the number of layers, early stopping of the learning process when the improvement of the fit becomes worse, etc.

We can use `history` variable to see the evolution of the loss function

In [ ]:
pd.DataFrame(history.history).plot(
figsize=(8, 5), xlim=[0, EPOCHS-1], ylim=[0, 1], grid=True, xlabel="Epoch", style=["r--", "r--.", "b-", "b-*"])

Even better!

# Superconductor dataset

Let's try a more complex example, making a regression to predict the critical temperature of our superconductor dataser

In [ ]:
# read and transform the data

superconductors = pd.read_csv("data/superconductor.csv", delimiter=",")

X = superconductors.drop("critical_temp", axis=1).to_numpy()
y = superconductors["critical_temp"].copy().to_numpy()

# define the scaler
scaler = StandardScaler()

# train the scaler
scaler.fit(X)

# Scale the data

X = scaler.transform(X)

C = 30
pca = PCA(n_components=C)
pca.fit(X)

print(f'Total number of components used after PCA : {pca.n_components_}')

# Choose training and test sets

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.20, random_state=42)

# Transform the train and test sets

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

X_test_pca.shape, X_train_pca.shape


Let us try a NN, starting with a simple one

In [ ]:
# clear the session and the previous NN defined
clear_session()

In [ ]:
model = Sequential()
model.add(Input(shape=(30,)))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(1))

In [ ]:
# Change the learning rate of the optimizer, making it take smaller steps in the
# search of the minimum of the loss function
adam = Adam(learning_rate=0.001)

model.compile(loss='mse', optimizer=adam,  metrics=['mae'])


In [ ]:
# Train with more epochs, but increasing the batch size to speed up the process
EPOCHS = 200
history = model.fit(X_train_pca, y_train, epochs=EPOCHS, batch_size=128)

In [ ]:
pd.DataFrame(history.history).plot(
figsize=(8, 5), grid=True, xlabel="Epoch", style=["r--", "r--.", "b-", "b-*"])

plt.yscale("log")

Decreasese, but very slowly.

In [ ]:
# get the preditions of the model
y_pred = model.predict(X_test_pca)

plt.scatter(y_test, y_pred, edgecolors='g')

plt.plot(y_test, y_test, "--r")

plt.xlabel("Real Temperature")
plt.ylabel("Predicted Temperature")

Using kNN regression, we achieved a Mean Square Error = 5.9725. With this simple NN.

In [ ]:
print(f"Mean Square Error: {mean_absolute_error(y_test, y_pred):.5}")

Worse than kNN. Let us try to improve, with  more complex NN

In [ ]:
# clear the session and the previous NN defined
clear_session()

In [ ]:
model = Sequential()
model.add(Input(shape=(30,)))
model.add(Dense(128, activation='tanh'))
model.add(Dense(128, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(1))

In [ ]:
# Change the learning rate of the optimizer, making it take smaller steps in the
# search of the minimum of the loss function
adam = Adam(learning_rate=0.001)

model.compile(loss='mse', optimizer=adam,  metrics=['mae'])


In [ ]:
# Train with more epochs, but increasing the batch size to speed up the process
EPOCHS = 200
history = model.fit(X_train_pca, y_train, epochs=EPOCHS, batch_size=128)

In [ ]:
# get the preditions of the model
y_pred = model.predict(X_test_pca)

plt.scatter(y_test, y_pred, edgecolors='g')

plt.plot(y_test, y_test, "--r")

plt.xlabel("Real Temperature")
plt.ylabel("Predicted Temperature")

In [ ]:
print(f"Mean Square Error: {mean_absolute_error(y_test, y_pred):.5}")

On  par with kNN. A more refined network could possibly lead toa better regression

# Binary classification with a MLP

Let us train a simple multilayer perceptron for binary classification. 

## Diabetes dataset

We apply it to the diabetes dataset, which using simpler ML methods we achieved a good 73% accuracy

In [ ]:
# Read the dataset

dataset = pd.read_csv('data/diabetes_database.csv')

In [ ]:
# Extract features and labels as numpy arrays for convenience

X = dataset.drop("Outcome", axis=1).to_numpy()
y = dataset.Outcome.to_numpy()

# rescale the features
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [ ]:
# Choose train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, 
random_state=42)

X_train.shape, X_test.shape

In [ ]:
# clear the session and the previous NN defined
clear_session()

In [ ]:
# define the model all at once, starting with a simple NN with a single layer

model = Sequential()
model.add(Input(shape=(8,))) # input data of 8 dimensions
model.add(Dense(32, activation='sigmoid'))  # Let us use a 'sigmoid' activation
model.add(Dense(1, activation='hard_sigmoid')) # Activation 'hard_sigmoid', to ensure output in [0,1]

In [ ]:
# compile the neural network
# for a binary classification, we use as loss (cost) function 'binary_crossentropy'
# the optimizer (to minimize the cost function) is the stochastic gradient descent algorithm 'adam`
# collect and record as metrics the accuracy of the classification


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# train the neural network with the 'fit' method
# training takes places in epochs = number of passes of in the training dataset
# and batches = number of samples in the model withing an epoch in which the weights (parameters) of the NN are updated

EPOCHS = 200
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=16, validation_data=(X_test, y_test))

In [ ]:
# plot the history
pd.DataFrame(history.history).plot(
figsize=(8, 5), xlim=[0, EPOCHS-1], ylim=[0, 1], grid=True, xlabel="Epoch", style=["r--", "r--.", "b-", "b-*"])

In [ ]:
# print the accuracy score, or fraction of fraction of correctly classified samples

predictions = (model.predict(X_test) > 0.5).astype(int)
print(f"Accuracy on Test Set: {accuracy_score(y_test, predictions) * 100:.2f}%")

Accuracy is larger than simple logistic regression for the test set


In [ ]:
# Confusion matrix
# make probability predictions with the model using the test dataset

predictions = (model.predict(X_test) > 0.5).astype(int)

CM = confusion_matrix(y_test, predictions, normalize="true", labels = [0, 1])
disp = ConfusionMatrixDisplay(CM, display_labels=["Healthy", "Diabetic"])

disp.plot()

We have slightly improved the prediction: from an accuracy of 73% with logistic regression, to an accuracy of 76% with a very simple MLP.

Given this success, let us try to improve even more with a deeper NN

In [ ]:
# clear the session and the previous NN defined
clear_session()

In [ ]:
# define the model all at once
# make it deeper!

model = Sequential()
model.add(Input(shape=(8,))) # input data of 8 dimensions
model.add(Dense(64, activation='sigmoid'))  # Let us use a 'sigmoid' activation
model.add(Dense(32, activation='sigmoid'))  # Let us use a 'sigmoid' activation
model.add(Dense(1, activation='hard_sigmoid')) # Activation 'hard_sigmoid', to ensure output in [0,1]

In [ ]:
 #compile the neural network
# for a binary classification, we use as loss (cost) function 'binary_crossentropy'
# the optimizer (to minimize the cost function) is the stochastic gradient descent algorithm 'adam`
# make the  learning rate smaller
# collect and record as metrics the accuracy of the classification

adam=Adam(learning_rate=0.001)

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# train the neural network with the 'fit' method
# training takes places in epochs = number of passes of in the training dataset
# and batches = number of samples in the model withing an epoch in which the weights (parameters) of the NN are updated

EPOCHS = 200
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=16, validation_data=(X_test, y_test))

In [ ]:
# plot the history
pd.DataFrame(history.history).plot(
figsize=(8, 5), xlim=[0, EPOCHS-1], ylim=[0, 1], grid=True, xlabel="Epoch", style=["r--", "r--.", "b-", "b-*"])

In [ ]:
# print the accuracy score, or fraction of fraction of correctly classified samples

predictions = (model.predict(X_test) > 0.5).astype(int)
print(f"Accuracy on Test Set: {accuracy_score(y_test, predictions) * 100:.2f}%")

Making the network deeper we have made the result slightly worse: From 76% accuracy we went down to 75%

In [ ]:
# Confusion matrix
# make probability predictions with the model using the test dataset

predictions = (model.predict(X_test) > 0.5).astype(int)

CM = confusion_matrix(y_test, predictions, normalize="true", labels = [0, 1])
disp = ConfusionMatrixDisplay(CM, display_labels=["Healthy", "Diabetic"])

disp.plot()

We have increased the ratio of false negatives, make the prediction worse...

Conclusion: Building NN is an art, the rule of the deeper the better does not work. 

To tackle a problem, we must study many possibilities and variations, tweaking many hyperparameters (as well as depth) until we find the best solution.

## Higgs Boson dataset

Let us try now the Higgs boson dataset

There we got a rather bad 64% accuracy with logistic regression, and KNN was no better

In [ ]:
bosons = pd.read_csv("data/higgs.csv", delimiter=";")

X = bosons.drop("Event", axis=1).to_numpy()
y = bosons["Event"].copy().to_numpy()


In [ ]:
# rescale the data to avoid complications

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
# clear the session and the previous NN defined
clear_session()

In [ ]:
# start with a shallow netwokr

model = Sequential()
model.add(Input(shape=(28,))) # input data of 8 dimensions
model.add(Dense(64, activation='sigmoid'))  # Let us use a 'sigmoid' activation
model.add(Dense(1, activation='hard_sigmoid')) # Activation 'hard_sigmoid', to ensure output in [0,1]

The features have 28 dimensions.

In [ ]:
# compile the neural network
# for a binary classification, we use as loss (cost) function 'binary_crossentropy'
# use default optimizer

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# train the neural network with the 'fit' method
# training takes places in epochs = number of passes of in the training dataset
# and batches = number of samples in the model withing an epoch in which the weights (parameters) of the NN are updated

# first pass
EPOCHS = 100

# second PASS
# EPOCHS = 200
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=64, validation_data=(X_test, y_test))

In [ ]:
pd.DataFrame(history.history).plot(
figsize=(8, 5), xlim=[0, EPOCHS-1], ylim=[0, 1], grid=True, xlabel="Epoch", style=["r--", "r--.", "b-", "b-*"])

In [ ]:
# print the accuracy score, or fraction of fraction of correctly classified samples

predictions = (model.predict(X_test) > 0.5).astype(int)
print(f"Accuracy on Test Set: {accuracy_score(y_test, predictions) * 100:.2f}%")

In [ ]:
# Confusion matrix

# make probability predictions with the model using the test dataset

predictions = (model.predict(X_test) >= 0.5).astype(int)

CM = confusion_matrix(y_test, predictions, normalize="true", labels = [0, 1])
disp = ConfusionMatrixDisplay(CM, display_labels=["Non Higgs", "Higgs"])

disp.plot()

A little bit better, 66% vs 64% accuracy.
One possibility is to rerun it with more training time, since in history we saw that the accuracy was growing.
Let's do it, going back in the code:

In [ ]:
pd.DataFrame(history.history).plot(
figsize=(8, 5), xlim=[0, EPOCHS-1], ylim=[0, 1], grid=True, xlabel="Epoch", style=["r--", "r--.", "b-", "b-*"])

In this second run, the accuracy on the train set in increasing, as well as the loss:

In [ ]:
# print the accuracy score for the train set

predictions = (model.predict(X_train) > 0.5).astype(int)
print(f"Accuracy on Test Set: {accuracy_score(y_train, predictions) * 100:.2f}%")

76%, we are making a good job in classifying the train set:

In [ ]:
# Confusion matrix for the train set


predictions = (model.predict(X_train) >= 0.5).astype(int)

CM = confusion_matrix(y_train, predictions, normalize="true", labels = [0, 1])
disp = ConfusionMatrixDisplay(CM, display_labels=["Non Higgs", "Higgs"])

disp.plot()

But we can see that the loss of the validation (test) set is increasing, and that validation accuracy remains more or less the same, or slightly decreases:

In [ ]:
# print the accuracy score for the test set

predictions = (model.predict(X_test) > 0.5).astype(int)
print(f"Accuracy on Test Set: {accuracy_score(y_test, predictions) * 100:.2f}%")

In [ ]:
# Confusion matrix for the train set


predictions = (model.predict(X_test) >= 0.5).astype(int)

CM = confusion_matrix(y_test, predictions, normalize="true", labels = [0, 1])
disp = ConfusionMatrixDisplay(CM, display_labels=["Non Higgs", "Higgs"])

disp.plot()

Add another hidden layer

In [ ]:
# clear the session and the previous NN defined
clear_session()

In [ ]:
# start with a shallow netwokr

model = Sequential()
model.add(Input(shape=(28,))) # input data of 8 dimensions
model.add(Dense(64, activation='sigmoid'))  # Let us use a 'sigmoid' activation
model.add(Dense(32, activation='sigmoid'))  # Let us use a 'sigmoid' activation
model.add(Dense(1, activation='hard_sigmoid')) # Activation 'hard_sigmoid', to ensure output in [0,1]

In [ ]:
# compile the neural network
# for a binary classification, we use as loss (cost) function 'binary_crossentropy'
# the optimizer (to minimize the cost function) is the stochastic gradient descent algorithm 'adam" "sgd"
# collect and record as metrics the accuracy of the classification

adam=Adam(learning_rate=0.001)

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# train the neural network with the 'fit' method
# training takes places in epochs = number of passes of in the training dataset
# and batches = number of samples in the model withing an epoch in which the weights (parameters) of the NN are updated

EPOCHS = 100

history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=64, validation_data=(X_test, y_test))

In [ ]:
pd.DataFrame(history.history).plot(
figsize=(8, 5), xlim=[0, EPOCHS-1], ylim=[0, 1], grid=True, xlabel="Epoch", style=["r--", "r--.", "b-", "b-*"])

In [ ]:
# print the accuracy score for the train set

predictions = (model.predict(X_train) > 0.5).astype(int)
print(f"Accuracy on Train Set: {accuracy_score(y_train, predictions) * 100:.2f}%")

In [ ]:
# print the accuracy score for the test set

predictions = (model.predict(X_test) > 0.5).astype(int)
print(f"Accuracy on Test Set: {accuracy_score(y_test, predictions) * 100:.2f}%")

In [ ]:
# Confusion matrix for the train set


predictions = (model.predict(X_test) >= 0.5).astype(int)

CM = confusion_matrix(y_test, predictions, normalize="true", labels = [0, 1])
disp = ConfusionMatrixDisplay(CM, display_labels=["Non Higgs", "Higgs"])

disp.plot()

Slight improvement, but still a lot of false negatives...

Not always the best option is a NN, sometimes other options are better.

Indeed, for classification and regression, the most recommended model are 
**boosted trees**

The idea of boosted trees is that a group of weak learners can become strong when combined. In this cases, it combines decision trees.

A decision tree is a decision support recursive partitioning structure that uses a tree-like model of decisions and their possible consequences.

The best algorithm in this is gradient boosted trees, implemented in the python package `xgboost`

In [ ]:
from xgboost import XGBClassifier

In [ ]:
# let us define the model. It has very many hyperparameters that can be tuned, to reach 
# the best accuracy. We choose some, more or less at random.

gbrt = XGBClassifier(max_depth=250, n_estimators=250, learning_rate=0.01, subsample=0.25, device="cuda", tree_method="hist")


In [ ]:
# Let us train it with the train set
    
gbrt.fit(X_train, y_train)

In [ ]:
# print the accuracy score for the test set

predictions = gbrt.predict(X_train)

print(f"Accuracy on Train Set: {accuracy_score(y_train, predictions) * 100:.2f}%")


Excellent accuracy in the training set, much better than NN

In [ ]:
# Confusion matrix for the train set


predictions = gbrt.predict(X_train)

CM = confusion_matrix(y_train, predictions, normalize="true", labels = [0, 1])
disp = ConfusionMatrixDisplay(CM, display_labels=["Non Higgs", "Higgs"])

disp.plot()

It has learned the training set almost perfectly!!

Let us see what happens in the test set

In [ ]:
# print the accuracy score for the test set

predictions = gbrt.predict(X_test)

print(f"Accuracy on Test Set: {accuracy_score(y_test, predictions) * 100:.2f}%")

In [ ]:
# Confusion matrix for the train set

CM = confusion_matrix(y_test, predictions, normalize="true", labels = [0, 1])
disp = ConfusionMatrixDisplay(CM, display_labels=["Non Higgs", "Higgs"])

disp.plot()

Reasonable improvement over logistic regression or NN